This notebook is an early exploration creating DICOM WSI files.  Newer code is available integrated with the RMS segmentation model in rms-set-with-dicom-out.py

In [7]:
import numpy as np
from pydicom import dcmread
import gc

# read the probability map output by the segmentation
img_array = np.load('/media/clisle/Imaging/IDC/sample_data_from_pixelmed/PARNED-0BNNF4_B2_Q30/DCM_0_prob_map_seg_stack.npy')
print('read array of dimensions:', img_array.shape)
# Read SM Image data set from PS3.10 files on disk.  This will provide the 
# reference image size and other dicom header information 
image_file =  '/media/clisle/Imaging/IDC/sample_data_from_pixelmed/PARNED-0BNNF4_B2_Q30/DCM_0'
image_dataset = dcmread(str(image_file))

read array of dimensions: (9264, 9808, 5)


In [85]:

def _gray_to_labelmap(input_probs):
    index_map = (np.argmax(input_probs, axis=-1)*50).astype('uint8')
    height = input_probs.shape[0]
    width = input_probs.shape[1]
    heatmap = np.zeros((height, width, 1), np.float32)
    # Background
    heatmap[index_map == 0, 0] = input_probs[:, :, 0][index_map == 0]
    # Necrosis
    heatmap[index_map==50, 0] = input_probs[:, :, 1][index_map==50]
    # Stroma
    heatmap[index_map==100, 0] = input_probs[:, :, 2][index_map==100]
    # ERMS
    heatmap[index_map==150, 0] = input_probs[:, :, 3][index_map==150]
    # ARMS
    heatmap[index_map==200, 0] = input_probs[:, :, 4][index_map==200]
    heatmap[np.average(heatmap, axis=-1)==0, :] = 1.
    return heatmap


# saved as I tried to change the output
def _gray_to_color(input_probs):

    index_map = (np.argmax(input_probs, axis=-1)*50).astype('uint8')
    height = index_map.shape[0]
    width = index_map.shape[1]

    heatmap = np.zeros((height, width, 3), np.float32)

    # Background
    heatmap[index_map == 0, 0] = input_probs[:, :, 0][index_map == 0]
    heatmap[index_map == 0, 1] = input_probs[:, :, 0][index_map == 0]
    heatmap[index_map == 0, 2] = input_probs[:, :, 0][index_map == 0]

    # Necrosis
    heatmap[index_map==50, 0] = input_probs[:, :, 1][index_map==50]
    heatmap[index_map==50, 1] = input_probs[:, :, 1][index_map==50]
    heatmap[index_map==50, 2] = 0.

    # Stroma
    heatmap[index_map==100, 0] = 0.
    heatmap[index_map==100, 1] = input_probs[:, :, 2][index_map==100]
    heatmap[index_map==100, 2] = 0.

    # ERMS
    heatmap[index_map==150, 0] = input_probs[:, :, 3][index_map==150]
    heatmap[index_map==150, 1] = 0.
    heatmap[index_map==150, 2] = 0.

    # ARMS
    heatmap[index_map==200, 0] = 0.
    heatmap[index_map==200, 1] = 0.
    heatmap[index_map==200, 2] = input_probs[:, :, 4][index_map==200]

    heatmap[np.average(heatmap, axis=-1)==0, :] = 1.

    return heatmap


In [42]:
print('the first dimension is the background feature')
print(img_array[00:10,00:5,0])
print('--------')
img_array[00:10,00:5,1]

the first dimension is the background feature
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
--------


array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [44]:
print(img_array[4000:4010,3000:3005,0])
print('--------')
print(img_array[4000:4010,3000:3005,2])

[[0.09885323 0.01833964 0.85569054 0.9999988  0.5437823 ]
 [0.06999329 0.01857144 0.07199194 0.0496218  0.07931215]
 [0.29979065 0.04545905 0.02517503 0.02872984 0.03119877]
 [0.99999994 0.06653909 0.03455681 0.0292804  0.02864437]
 [0.18684249 0.99924254 0.05343835 0.02715843 0.06112269]
 [0.02161304 0.03637022 0.02607653 0.0231246  0.03877334]
 [0.02662762 0.03028561 0.01278969 0.03566855 0.02838792]
 [0.09287167 0.9999998  0.37906298 0.10865007 0.03947021]
 [0.99174863 0.9998394  0.0346764  0.99364436 0.02019441]
 [0.06616805 0.01758871 0.03465229 0.20315011 0.01553984]]
--------
[[1.5822272e-03 1.5085234e-03 5.4091460e-04 3.9947037e-09 7.7487825e-04]
 [1.7646151e-03 1.3401026e-03 1.2013029e-03 1.2899204e-03 1.0527071e-03]
 [1.8518667e-03 1.6911210e-03 1.2253837e-03 1.4700033e-03 1.5215054e-03]
 [4.4405285e-10 2.9537315e-03 1.6524281e-03 1.7145807e-03 2.2892633e-03]
 [2.7323849e-03 1.7574280e-06 2.3762167e-03 2.0051009e-03 2.2525184e-03]
 [2.5277080e-03 1.5831784e-03 1.6642596e-03 1

In [10]:
NUM_CLASSES = 5
CLASS_VALUES = [0, 50, 100, 150, 200]

In [11]:
pred_map_final = np.argmax(img_array, axis=-1)
pred_map_final_gray = pred_map_final.astype('uint8') * 50
#del pred_map_final
#gc.collect()
pred_map_final_ones = [(pred_map_final_gray == v) for v in CLASS_VALUES]
#del pred_map_final_gray
#gc.collect()
pred_map_final_stack = np.stack(pred_map_final_ones, axis=-1).astype('uint8')
#del pred_map_final_ones
#gc.collect()

pred_colormap = _gray_to_color(pred_map_final_stack)

In [89]:
test_img = np.argmax(img_array, axis=-1)
test_img[4000:4010,3000:3005]

array([[3, 3, 0, 0, 0],
       [3, 3, 3, 3, 3],
       [3, 3, 3, 3, 3],
       [0, 3, 3, 3, 3],
       [3, 0, 3, 3, 3],
       [3, 3, 3, 3, 3],
       [3, 3, 3, 3, 3],
       [3, 0, 3, 3, 3],
       [0, 0, 3, 0, 3],
       [3, 3, 3, 3, 3]])

In [14]:
print(pred_map_final_gray.shape)
pred_map_final_gray[4000:4010,3000:3005]

(9264, 9808)


array([[150, 150,   0,   0,   0],
       [150, 150, 150, 150, 150],
       [150, 150, 150, 150, 150],
       [  0, 150, 150, 150, 150],
       [150,   0, 150, 150, 150],
       [150, 150, 150, 150, 150],
       [150, 150, 150, 150, 150],
       [150,   0, 150, 150, 150],
       [  0,   0, 150,   0, 150],
       [150, 150, 150, 150, 150]], dtype=uint8)

In [18]:
print(pred_map_final_ones)

[array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]]), array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]]), array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       

In [33]:
print('this map is separated out into separate segments:')
print(pred_map_final_stack.shape)
print(pred_map_final_stack[4000:4010,3000:3005,0])
print('-------')
print(pred_map_final_stack[4000:4010,3005:3010,3])

this map is separated out into separate segments:
(9264, 9808, 5)
[[0 0 1 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 1 0 0 0]
 [1 1 0 1 0]
 [0 0 0 0 0]]
-------
[[1 1 1 1 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 0 1]]


In [34]:
print('this map is separated out into separate segments:')
print('The first dimension is all background:')
print(pred_map_final_stack.shape)
print(pred_map_final_stack[00:10,00:5,0])
print('-------')
print(pred_map_final_stack[0:10,5:10,3])

this map is separated out into separate segments:
(9264, 9808, 5)
[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
-------
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


Since we have five classes, we want to strip off one of them before passing to the dicom conversion routine, since it is expecting a four channel array. According to the HighDicom documentation (https://highdicom.readthedocs.io/en/latest/package.html#module-highdicom.seg), A 4D array will be expected in the format: [NumberOfFrames,Row, Columns, NumOfSegments].  So this needs to be reformatted, and there is a routine in dicomslide for this already.

In [84]:
# segs in order: Bground, Necrosis, Stroma, ARMS, ERMS
# picking ERMS
intermediate = pred_map_final_stack[:,:,4]
print('shape of intermediate:',intermediate.shape)
# flip the rows and columns to match dicom ordering
#image_to_import = np.rollaxis(intermediate,1)
print('shape of image_to_import:',intermediate.shape)
intermediate.shape


shape of intermediate: (9264, 9808)
shape of image_to_import: (9264, 9808)


(9264, 9808)

In [67]:

from pathlib import Path

import highdicom as hd
import dicomslide
from pydicom.sr.codedict import codes
from pydicom.filereader import dcmread
from pydicom import Dataset
from dicomweb_client import DICOMfileClient
from tempfile import TemporaryDirectory
from typing import Tuple

def disassemble_total_pixel_matrix_wrapped(
    seg_total_pixel_matrix: np.ndarray,
    source_image_metadata: Dataset,
) -> np.ndarray:
    """Disassemble a total pixel matrix into individual tiles.

    Parameters
    ----------
    seg_total_pixel_matrix: numpy.ndarray
        Total pixel matrix of the segmentation as a 2D NumPy array.
    source_metadata: pydicom.Dataset
        DICOM metadata of the source image.

    Returns
    -------
    numpy.ndarray
        Stacked image tiles

    """
    if seg_total_pixel_matrix.ndim != 2:
        raise ValueError(
            "Total pixel matrix has unexpected number of dimensions."
        )

    # Need a client object to work with DICOM slide so just create a dummy one
    with TemporaryDirectory() as tmpdir:
        client = DICOMfileClient(f"file://{tmpdir}")

        im_tpm = dicomslide.TotalPixelMatrix(client, source_image_metadata)

        tile_rows, tile_cols, _ = im_tpm.tile_shape

        return dicomslide.disassemble_total_pixel_matrix(
            seg_total_pixel_matrix,
            im_tpm.tile_positions,
            tile_rows,
            tile_cols,
        )














In [82]:
#  This is based on the hidicom output example listed in the readthedocs documentation

CHANNEL_DESCRIPTION = {}
CHANNEL_DESCRIPTION['chan_1'] = 'Necrosis'
CHANNEL_DESCRIPTION['chan_2'] = 'Stroma'
CHANNEL_DESCRIPTION['chan_3'] = 'ARMS'
CHANNEL_DESCRIPTION['chan_4'] = 'ERMS'

def writeDicomSegObject(image_dataset, seg_image, out_path):

    # Path to multi-frame SM image instance stored as PS3.10 file
    #image_file = Path(image_path)

    # Read SM Image data set from PS3.10 files on disk.  This will provide the 
    # reference image size and other dicom header information
    #image_dataset = dcmread(str(image_file))


    # make the derived image header information (from Max)
    # this has to be run expecting a single image
    derived_plane_positions,derived_pixel_measures = _compute_derived_image_attributes(image_dataset, seg_image)

   # function stolen from idc-pan-cancer-archive repository to re-tile the numpy to match the tiling
   # from the source image
    print('passing in a numpy array of shape:',seg_image.shape)
    # pass only the ERMS channel?
    mask = disassemble_total_pixel_matrix_wrapped(seg_image,image_dataset)
    print('received a numpy array of shape:',mask.shape)


    # Describe the algorithm that created the segmentation
    algorithm_identification = hd.AlgorithmIdentificationSequence(
        name='FNLCR_RMS_seg_iou_0.7343_epoch_60',
        version='v1.0',
        family=codes.cid7162.ArtificialIntelligence
    )

    # Describe the segment
    description_segment_1 = hd.seg.SegmentDescription(
        segment_number=1,
        segment_label=CHANNEL_DESCRIPTION['chan_4'],
        segmented_property_category=codes.cid7150.Tissue,
        segmented_property_type=codes.cid7166.ConnectiveTissue,
        algorithm_type=hd.seg.SegmentAlgorithmTypeValues.AUTOMATIC,
        algorithm_identification=algorithm_identification,
        tracking_uid=hd.UID(),
        tracking_id='RMS segmentation_'+str(CHANNEL_DESCRIPTION['chan_4'])
    )



    # Create the Segmentation instance
    seg_dataset = hd.seg.Segmentation(
        source_images=[image_dataset],
        pixel_array=mask,
        segmentation_type=hd.seg.SegmentationTypeValues.BINARY,
        #segment_descriptions=[description_segment_1,description_segment_2,description_segment_3,description_segment_4],
        segment_descriptions=[description_segment_1],
        series_instance_uid=hd.UID(),
        series_number=2,
        sop_instance_uid=hd.UID(),
        instance_number=1,
        # the following two entries are added because the output resolution is different from the source
        pixel_measures=derived_pixel_measures,
        plane_positions= derived_plane_positions,
        manufacturer='Aperio',
        manufacturer_model_name='Unknown',
        software_versions='v1',
        device_serial_number='Unknown'
    )

    print(seg_dataset)
    # change output file with some function is needed
    outfilename = out_path
    seg_dataset.save_as(outfilename)



In [73]:
from pydicom import Dataset

# from Max to calculate the derived parameters needed when the segmentation image is not the same
# resolution as the source image.  This creates records that have to be passed in to the creation
# step in HighDicom. 

def _compute_derived_image_attributes(source_image: Dataset, total_pixel_matrix: np.ndarray) -> Tuple[hd.PlanePositionSequence, hd.PixelMeasuresSequence]:
    """Compute attributes of a derived single-frame image.
    Parameters
    ----------
    source_image: pydicom.Dataset
        Source image from which single-frame image was derived
    total_pixel_matrix: numpy.ndarray
        Total Pixel matrix of derived single-frame image for which attribute
        values should be computed
    Returns
    -------
    plane_positions: highdicom.PlanePositionSequence
        Plane position of the single-frame image
    pixel_measures: highdicom.PixelMeasuresSequence
        Pixel measures of the single-frame image
    """
    sm_total_rows = int(
        np.ceil(source_image.TotalPixelMatrixRows / source_image.Rows)
        * source_image.Rows
    )
    sm_total_cols = int(
        np.ceil(source_image.TotalPixelMatrixColumns / source_image.Columns)
        * source_image.Columns
    )
    print('sm_total_rows=',sm_total_rows)
    print('sm_total_cols=',sm_total_cols)
    origin = source_image.TotalPixelMatrixOriginSequence[0]
    print('origin=',origin)
    x_offset = origin.XOffsetInSlideCoordinateSystem
    y_offset = origin.YOffsetInSlideCoordinateSystem
    print('x-offset:',x_offset,'y_offset:',y_offset)
    sm_shared_func_groups = source_image.SharedFunctionalGroupsSequence[0]
    sm_pixel_measures = sm_shared_func_groups.PixelMeasuresSequence[0]
    print('sm_pixel_measures:',sm_pixel_measures)
    sm_pixel_spacing = sm_pixel_measures.PixelSpacing
    print('sm_pixel_spacing:',sm_pixel_spacing)
    sm_slice_thickness = sm_pixel_measures.SliceThickness
    print('sm_slice_thickness:',sm_slice_thickness)
    print('total_pixel_matrix_shape:',total_pixel_matrix.shape)
    derived_pixel_spacing = (
        (sm_total_rows * sm_pixel_spacing[0]) / total_pixel_matrix.shape[0],
        (sm_total_cols * sm_pixel_spacing[1]) / total_pixel_matrix.shape[1],
    )
    print('derived_pixel_spacing:',derived_pixel_spacing)
    derived_plane_position = hd.PlanePositionSequence(
        coordinate_system=hd.CoordinateSystemNames.SLIDE,
        image_position=(x_offset, y_offset, 0.0),
        pixel_matrix_position=(1, 1),  # there is only one frame
    )
    print('derived_plane_position:',derived_plane_position)
    derived_pixel_measures = hd.PixelMeasuresSequence(
        pixel_spacing=derived_pixel_spacing, slice_thickness=sm_slice_thickness
    )
    print('derived_pixel measures:',derived_pixel_measures)
    return (derived_plane_position, derived_pixel_measures)



In [83]:
outpath = '/media/clisle/Imaging/IDC/sample_data_from_pixelmed/PARNED-0BNNF4_B2_Q30/DCM_0_from_numpy.dcm'
writeDicomSegObject(image_dataset,image_to_import,outpath)

sm_total_rows= 73472
sm_total_cols= 77824
origin= (0040, 072a) X Offset in Slide Coordinate System DS: '22.362505'
(0040, 073a) Y Offset in Slide Coordinate System DS: '32.4148281116585'
x-offset: 22.362505 y_offset: 32.4148281116585
sm_pixel_measures: (0018, 0050) Slice Thickness                     DS: '0.0'
(0028, 0030) Pixel Spacing                       DS: [.0002526, .0002526]
sm_pixel_spacing: [.0002526, .0002526]
sm_slice_thickness: 0
total_pixel_matrix_shape: (9808, 9264)
derived_pixel_spacing: (0.0018922336052202286, 0.002122014507772021)
derived_plane_position: [(0040, 072a) X Offset in Slide Coordinate System DS: '22.362505'
(0040, 073a) Y Offset in Slide Coordinate System DS: '32.4148281116585'
(0040, 074a) Z Offset in Slide Coordinate System DS: '0.0'
(0048, 021e) Column Position In Total Image Pixe SL: 1
(0048, 021f) Row Position In Total Image Pixel M SL: 1]
derived_pixel measures: [(0018, 0050) Slice Thickness                     DS: '0'
(0028, 0030) Pixel Spacing     

/home/clisle/proj/slicer/PW39/highdicom/src/highdicom/valuerep.py:54: UserWarning: The string "RMS2341" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in http://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  warnings.warn(


ValueError: Number of PlanePositionSequence items provided via "plane_positions" argument does not match size of first dimension of "pixel_array" argument.